#### Random Graph Generation

In [1]:
import numpy as np
import random
import pickle
import os

K=[10]
nV=[100]                      #nv is the number of vertices in the graph.  
nC=[10]                      #cv is the number of clients for each facility
num_graphs=5
penalty=10000

modes=['skewed[1-10]']
da=['training','testing']
# mode='/skewed[1-6]'
# mode='/skewed[1,10]'
for nv in nV:
    for nc in nC:
        for mode in modes:
            os.system('mkdir'+' data/')
            os.system('mkdir'+' data/'+mode)
            os.system('mkdir'+' data/'+mode+'/V='+str(nv)+'-C='+str(nc))
            for ki,k in enumerate(K):
                os.system('mkdir'+' data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k))
                os.system('mkdir'+' data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/testing')
                os.system('mkdir'+' data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/training')
                print(nv,nc,mode,k)
                for d in da:
                    path='data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/'+d+'/'
                    for graph in range(num_graphs):
                        adj=np.zeros([nv*nc,3])
                        new_pairs=[]
                        for i in range(1,nv+1):
                            y=np.sort(random.sample(range(1,nv+1), nc))
                            for j in (y):
                                new_pairs.append((i,j))
                        new_pairs=np.array(new_pairs)


                        adj[:,0]=new_pairs[:,0]
                        adj[:,1]=new_pairs[:,1]
                        #adj[:,2]= random.choices([1,10], weights = [3, 6], k = nc*nv)
                        #adj[:,2]np.random.randint(low = 1,high=6, size=nc*nv)
                        adj[:,2]= random.choices([1,2,3,4,5,6,7,8,9,10], weights = [1,1,2,2,3,3,4,4,5,5], k = nc*nv)
                        #adj[:,2]= random.choices([1,100], k = nc*nv)

                        os.system("mkdir "+path+str(graph)); pickle.dump([adj,nv,k],open(path+str(graph)+'/data','wb'))
                        np.savetxt(path+str(graph)+'/data.txt', adj, fmt='%10d' ,header=str(nv)+'   '+str(nv*nc)+'   '+str(k))

100 10 skewed[1-10] 10


#### LP for feature computation

In [2]:
import xpress as xp
from termcolor import colored
import timeit
import numpy as np
import pickle
import random
def kmedian_xpress_solver_LP(set_path,data,c,n,k):
    print("Inside kmedian",n,k)
    S1 = timeit.default_timer()
    x = np.array ([xp.var (vartype = xp.continuous,ub=1.0) for i in range(n*n)]).reshape (n,n)
    y = np.array([xp.var (vartype = xp.continuous,ub=1.0) for i in range(n)])
    #x = np.array ([xp.var (vartype = xp.free) for i in range(n*n)]).reshape (n,n)
    #y = np.array([xp.var (vartype = xp.free) for i in range(n)])
    print(x.shape,y.shape)
    
    c1=[xp.Sum(y)==k]
    c2=xp.Dot(x,np.ones([n,1]))==np.ones([n,1])
    c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,n),x],axis=0))<=np.zeros([n,n])

    #c4=0<=x; c5=x<=1;  c6=0<=y;  c7=y<=1

    profit=xp.Sum(xp.Sum(c*x))

    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.minimize)
    try:
        p.addConstraint(c1,c2,c3)#,c4,c5,c6,c7)
    except xp.ModelError as e:
      print ("Modeling error:", repr(e))
    p.solve()
    y=p.getSolution(y)
    x=p.getSolution(x)
    E1 = timeit.default_timer()
    #################################################################################################
    median_indices=np.where(y>0.0)[0]
    print(colored('            Time: ','red'), E1 - S1) 
    print(colored('            objective value: ','red'),p.getSolution(profit))
    #print(colored('median indices: ','red'),median_indices+1,median_indices.shape)
     #------------------------------------------------------
    pickle.dump([median_indices,p.getSolution(profit),E1-S1],open(set_path+'/lp_inf','wb'))
    return median_indices,x,y

#### Computing LP based features

In [3]:
def features_related_LP(set_path,median_indices,x,y,c,n):
    #------------------lp_kmedian----------------------
    ind=np.zeros(n);ind[median_indices]=1
    pickle.dump(ind,open(set_path+'/lp_kmedian','wb'))
    pickle.dump(y,open(set_path+'/yf','wb'))
    #--------------------alfa_f-------------------------
    alfa_f=np.zeros(n)
    for i in range(n):
        alfa_f[i]= sum(x[:,i])
    pickle.dump(alfa_f,open(set_path+'/alfa_f','wb'))
    #---------------------µf-------------------------
    µf=np.zeros(n);
    for i in range(n):
        if alfa_f[i]!=0:
            µf[i]= sum(x[:,i]*c[:,i])/alfa_f[i]
    pickle.dump(µf,open(set_path+'/µf','wb'))
    #---------------------Vf,t(t=2,4,8)-------------------------
    T=[2,4,8]
    for t in T:
        V_ft=np.zeros(n);
        for i in range(n):
            V_ft[i]=len(np.where(c[:,i]<=(t*µf[i]))[0])
        pickle.dump(V_ft,open(set_path+'/V_f_'+str(t),'wb'))
    #---------------------Bf,t(t=2,4,8)-------------------------
    dav=np.zeros(n)
    for i in range(n):
        dav[i]=sum(c[:,i]*x[:,i])

    T=[2,4,8]
    for t in T:
        B_ft=np.zeros(n)
        for i in range(n):
            B_ft[i]=len(np.where(c[:,i]<=(t*dav[i]))[0])
        pickle.dump(B_ft,open(set_path+'/B_f_'+str(t),'wb'))
                    
    #----------------------------Yf----------------------------
    Yf=np.zeros(n)
    for i in range(n):
        Yf[i]=(len(np.where(x[:,i]!=0)[0]))
    pickle.dump(Yf,open(set_path+'/Yf','wb'))

#### Computing features not based on LP

In [4]:
def features_no_related_LP(Mypath,cost_mtr,nv):
    #--------------------------------------------------------
    degree=np.zeros(nv)
    for j in range(nv):
        s=np.where(cost_mtr[:,j]!=penalty)[0]
        degree[j]=sum(cost_mtr[s,j])
    pickle.dump(degree,open(Mypath+'/degree','wb'))
    #--------------------------------------------------------
    T=[2,4,8]
    for t in T:
        T_ft=np.zeros(nv);
        for i in range(nv):
            T_ft[i]=len(np.where(cost_mtr[:,i]<=t)[0])

        pickle.dump(T_ft,open(Mypath+'/T_f_'+str(t),'wb'))
    #--------------------------------------------------------
    min_c=np.zeros(nv)
    for i in range(nv):
        min_c[i]=min(cost_mtr[:,i])
    pickle.dump(min_c,open(Mypath+'/min_c','wb'))
    #---------------------------min_f-----------------------------
    min_f=np.zeros(nv)
    for i in range(nv):
        min_f[i]=min(cost_mtr[:,i])
    pickle.dump(min_f,open(Mypath+'/min_f','wb'))
    #----------------------------Yf,p=1.5----------------------------
    Yf=np.zeros(nv)
    for i in range(nv):
        Yf[i]=len(np.where(cost_mtr[:,i]<=(1.5*min_f[i]))[0])
    pickle.dump(Yf,open(Mypath+'/Yfp','wb'))
    #------------------------------------------------------------------------------------
    s=np.sort(random.sample(range(nv), int(nv/2)))
    s1=np.zeros(nv);s1[s]=1
    ss=[]
    for i in range(nv):
        if i not in s:
            ss.append(i)
    ss=np.array(ss);s2=np.zeros(nv);s2[ss]=1
    pickle.dump(s1,open(Mypath+'/f1','wb'))
    pickle.dump(s2,open(Mypath+'/f2','wb'))

#### Calling feature computation (LP-based and non LP-based) after computing LP

In [5]:
for Mode in modes:
    print(Mode,'--------------------------')
    for nv in nV:
        for nc in nC:
            print('------------------------------------------')
            for k in K:
                print('------------------------------------------')
                for d in da:
                    print('--- nV=',nv,' nC=',nc,' k=',k, ' mode=',d)
                    for graph in range(num_graphs):
                        print('       data=',graph)
                        Mypath='data/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/'+d+'/'+str(graph)
                        data = pickle.load(open(Mypath+'/data','rb'))
                        nv= data[1]; k=data[2]; data=data[0]
                        data=data.astype(int)
                        data[:,0]=data[:,0]-1
                        data[:,1]=data[:,1]-1
                        cost_mtr=np.zeros([nv,nv])+penalty
                        for i in range (len(data)):
                            cost_mtr[data[i,1],data[i,0]]=data[i,2]
                        #print("Calling kmedian with ",Mypath,data.shape,cost_mtr.shape,nv,k)
                        median_indices,x,y=kmedian_xpress_solver_LP(Mypath,data,cost_mtr,nv,k)
                        features_related_LP(Mypath,median_indices,x,y,cost_mtr,nv)
                        features_no_related_LP(Mypath,cost_mtr,nv)

skewed[1-10] --------------------------
------------------------------------------
------------------------------------------
--- nV= 100  nC= 10  k= 10  mode= training
       data= 0
Inside kmedian 100 10
(100, 100) (100,)
FICO Xpress v8.10.0, Hyper, solve started 14:39:52, Aug 13, 2021
Heap usage: 5146KB (peak 5146KB, 812KB system)
Minimizing LP noname
Original problem has:
     10101 rows        10100 cols        30100 elements
Presolved problem has:
     10101 rows        10100 cols        30100 elements
Presolve finished in 0 seconds
Heap usage: 6841KB (peak 10MB, 814KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+01] / [ 1.00e+00,  1.00e+01]
  Objective      [min,max] : [ 1.00e+00,  1.00e+04] / [ 1.00e+00,  1.00e+04]
Autoscaling applied standard scaling

Crash basis containing 100 structural columns created
Starting 

  1941       43786.48087      P      0     0        .000000     0
Uncrunching matrix
Optimal solution found
Dual solved problem
  1941 simplex iterations in 0s

Final objective                       : 4.378648087021003e+04
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max dual violation        (abs/rel) :       0.0 /       0.0
  Max complementarity viol. (abs/rel) :       0.0 /       0.0
            Time:  0.623698138166219
            objective value:  43786.48087021005

       data= 1
Inside kmedian 100 10
(100, 100) (100,)
FICO Xpress v8.10.0, Hyper, solve started 14:39:56, Aug 13, 2021
Heap usage: 5146KB (peak 5146KB, 852KB system)
Minimizing LP noname
Original problem has:
     10101 rows        10100 cols        30100 elements
Presolved problem has:
     10101 rows        10100 cols        30100 elements
Presolve finished in 0 seconds
Heap usage: 6841KB (peak 10MB, 854KB system)

Coefficient range                    original                 solved        
  Coef

#### Run ILP to get labelled training data and evaluation on test data

In [6]:
import xpress as xp
from termcolor import colored
import timeit
import math
import numpy as np
import pickle
import sys
import matplotlib.pyplot as plt
import networkx as nx

def floyd_warshall(G,nV):
    distance = list(map(lambda i: list(map(lambda j: j, i)), G))

    # Adding vertices individually
    for k in range(nV):
        for i in range(nV):
            for j in range(nV):
                distance[i][j] = min(distance[i][j], distance[i][k] + distance[k][j])
    return (distance)

def kmedian_xpress_solver_ILP(data,n,k):

    c=np.zeros([n,n])+penalty
    data=data.astype(int)
    data[:,0]=data[:,0]-1
    data[:,1]=data[:,1]-1
    for i in range (len(data)):
        c[data[i,1],data[i,0]]=data[i,2]
    #c=floyd_warshall(c,n)

    S=range(n);SS=range(n*n)
    x = np.array ([xp.var (vartype = xp.binary) for i in SS]).reshape (n,n)
    y = np.array([xp.var (vartype = xp.binary) for i in S])

    c1=[xp.Sum(y)==k]
    c2=xp.Dot(x,np.ones([n,1]))==np.ones([n,1])
    c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,n),x],axis=0))<=np.zeros([n,n])

    profit=xp.Sum(xp.Sum(c*x))

    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.minimize)
    p.addConstraint(c1,c2)
    p.addConstraint(c3)
    p.solve()

    median_indices=np.where(p.getSolution(y)>=.5)[0]
    return median_indices,p.getSolution(profit)


for nv in nV:
    for nc in nC:
        for k in K:
            for mode in modes:
                for d in da:
                    path='data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/'+d+'/'
                    for m in range(num_graphs):
                        print(colored('....................................................','blue'),m)
                        S = timeit.default_timer()
                        set_path=path+str(m)
                        data = pickle.load(open(set_path+'/data','rb'))
                        #print(data[0],data[1],data[2])

                        labels=kmedian_xpress_solver_ILP(data[0],data[1],data[2])

                        E = timeit.default_timer() 
                        pickle.dump([labels,E-S],open(set_path+'/points_inf_xp','wb'))

                        print(data[1],data[2],data[0].shape)
                        print(colored('objective value: ','red'),labels[1])
                        print(E - S) 
                        print(colored('median indices: ','red'),labels[0]+1,len(labels[0]))

.................................................... 0
FICO Xpress v8.10.0, Hyper, solve started 14:40:25, Aug 13, 2021
Heap usage: 5445KB (peak 5445KB, 3038KB system)
Minimizing MILP noname
Original problem has:
     10101 rows        10100 cols        30100 elements     10100 globals
Presolved problem has:
     10101 rows        10100 cols        30100 elements     10100 globals
Presolve finished in 0 seconds
Heap usage: 8637KB (peak 12MB, 3039KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+01] / [ 1.00e+00,  1.00e+01]
  Objective      [min,max] : [ 1.00e+00,  1.00e+04] / [ 1.00e+00,  1.00e+04]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 88.3GB
 *** Heuristic solution found:  900067.0000      Time: 0 ***
Starting concurrent solve with dual, primal and barrier (30 thread

 
   Its         Obj Value      S   Ninf  Nneg        Sum Inf  Time
1194204       37435.87039      P      0     0        .000000    31
Optimal solution found
Dual solved problem
  1194204 simplex iterations in 31s

Final objective                       : 3.743587038515593e+04
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max dual violation        (abs/rel) :       0.0 /       0.0
  Max complementarity viol. (abs/rel) :       0.0 /       0.0
 
Starting root cutting & heuristics
 
 Its Type    BestSoln    BestBound   Sols    Add    Del     Gap     GInf   Time
   1  K   150550.0000  74154.40566      8      4      0   50.74%     786     38
   2  K   150550.0000  74154.40566      8      3      5   50.74%     709     38
   3  K   150550.0000  74154.40566      8      2      3   50.74%     759     39
   4  K   150550.0000  74154.40566      8      2      0   50.74%     766     39
   5  K   150550.0000  74154.40566      8      2      5   50.74%     765     40
   6  K   150550.0

    8938  150550.0000  79810.08395      8   3586     25   46.99%     461    131
    9040  150550.0000  79810.08395      8   3628     29   46.99%     496    131
    9144  150550.0000  79810.08395      8   3647     22   46.99%     507    132
    9244  150550.0000  79810.08395      8   3675     20   46.99%     441    132
Heap usage: 1903MB (peak 1903MB, 19MB system)
B&B tree size: 71MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    9344  150550.0000  79810.08395      8   3721     42   46.99%     415    132
    9444  150550.0000  79810.08395      8   3744     33   46.99%     490    133
    9548  150550.0000  79810.08395      8   3759     16   46.99%     366    133
    9657  150550.0000  79810.08395      8   3829     40   46.99%     556    133
    9760  150550.0000  79810.08395      8   3865     21   46.99%     293    134
    9861  150550.0000  79810.08395      8   3899     39   46.99%     536    134
    9965  150550.0000  81394.55954      8   39


                           Concurrent-Solve,   0s
            Dual                      Primal                     Barrier      
                                                          objective   sum inf 
                         |                          |  P  46459.850   .0000000
----- interrupted ------ | ----- interrupted ------ | ------- optimal --------
Concurrent statistics:
      Dual: 1331 simplex iterations, 0.14s
    Primal: 1867 simplex iterations, 0.14s
   Barrier: 16 barrier and 32 simplex iterations, 0.14s
            Barrier used 30 threads 16 cores, L1\L2 cache: 32K\32768K
            Barrier used AVX support, crossover used 8 threads
Optimal solution found
 
   Its         Obj Value      S   Ninf  Nneg        Sum Inf  Time
     0       46459.84978      P      0     0        .000000     0
Barrier solved problem
  16 barrier iterations in 0s

Final objective                       : 4.645984977882717e+04
  Max primal violation      (abs/rel) :       0.0 /       0.0


   5  K   170525.0000  75581.36669     13      2      3   55.68%     715     42
   6  K   170525.0000  75581.36669     13      3      3   55.68%     726     42
   7  K   170525.0000  75581.36669     13      1      1   55.68%     743     42
   8  K   170525.0000  75581.36669     13      0      1   55.68%     732     43
Heuristic search started
Heuristic search stopped
 
Cuts in the matrix         : 11
Cut elements in the matrix : 39490
 
Starting tree search.
Deterministic mode with up to 32 running threads and up to 64 tasks.
Heap usage: 31MB (peak 1752MB, 4361KB system)
Heap usage: 31MB (peak 1752MB, 4361KB system)
B&B tree size: 98k total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    3078  170525.0000  75581.36669     13      2      1   55.68%     732     53
    3079  170525.0000  75581.36669     13      2      3   55.68%     701     54
    3080  170525.0000  75581.36669     13      2      3   55.68%     658     54
    3081  170525.0000  75581.

...

   12399  140570.0000  100225.5547     12    161     16   28.70%     338    141
   12499  140570.0000  100225.5547     12    140     20   28.70%     442    141
   12600  140570.0000  100225.5547     12    129     16   28.70%     458    142
   12700  140570.0000  100801.6947     12    122     18   28.29%     339    142
   12801  140570.0000  103222.1827     12    109     33   26.57%     495    142
   12903  140570.0000  103222.1827     12    101     12   26.57%     366    143
   13003  140570.0000  107436.9189     12     74     38   23.57%     423    143
   13104  140570.0000  107436.9189     12     73     26   23.57%     480    144
   13204  140570.0000  107436.9189     12     64     16   23.57%     342    144
 *** Search completed ***
Uncrunching matrix
Final MIP objective                   : 1.405700000000000e+05
Final MIP bound                       : 1.405700000000000e+05
  Solution time / primaldual integral :       145s/ 52.044644%
  Number of solutions found / nodes   :        1

    1937  170529.0000  68375.90134      7   1396     47   59.90%     596     34
    2038  170529.0000  68375.90134      7   1438     66   59.90%     463     35
    2141  170529.0000  68375.90134      7   1451     38   59.90%     457     35
    2241  170529.0000  68375.90134      7   1554     56   59.90%     448     36
    2342  170529.0000  70479.73756      7   1672     31   58.67%     450     36
    2442  170529.0000  70479.73756      7   1726     20   58.67%     331     36
    2542  170529.0000  70479.73756      7   1798     46   58.67%     551     36
b   2638  170504.0000  70479.73756      8   1893     36   58.66%       0     37
    2643  170504.0000  70479.73756      8   1893     41   58.66%     516     37
    2743  170504.0000  70479.73756      8   1988     34   58.66%     528     37
    2845  170504.0000  70479.73756      8   2108     50   58.66%     427     38
    2945  170504.0000  70479.73756      8   2141     21   58.66%     521     38
    3046  170504.0000  70479.73756      

    7342  140536.0000  77635.19729     10   2133     35   44.76%     443    121
    7442  140536.0000  79321.07790     10   2155     15   43.56%     348    122
    7543  140536.0000  79321.07790     10   2216     30   43.56%     425    122
    7649  140536.0000  79321.07790     10   2239     36   43.56%     523    122
    7751  140536.0000  79321.07790     10   2253     21   43.56%     438    122
    7852  140536.0000  79321.07790     10   2279     19   43.56%     409    123
    7952  140536.0000  79321.07790     10   2297     13   43.56%     226    123
    8055  140536.0000  79321.07790     10   2296     25   43.56%     522    124
    8155  140536.0000  79321.07790     10   2313     30   43.56%     554    124
    8258  140536.0000  79321.07790     10   2336     31   43.56%     554    124
    8359  140536.0000  79321.07790     10   2364     22   43.56%     567    124
    8463  140536.0000  79321.07790     10   2396     18   43.56%     464    125
    8563  140536.0000  79321.07790     1

   5  K   220499.0000  49839.22149      4      2      3   77.40%     753      2
   6  K   220499.0000  49949.17403      4      2      2   77.35%     734      2
   7  K   220499.0000  50004.94279      4      2      3   77.32%     731      3
   8  K   220499.0000  50050.28205      4      2      0   77.30%     760      3
   9  K   220499.0000  50052.79918      4      1      2   77.30%     777      3
  10  K   220499.0000  50052.79918      4      0      0   77.30%     779      4
Heuristic search started
R         220494.0000  50052.79918      5                 77.30%       0      9
Heuristic search stopped
 
Cuts in the matrix         : 8
Cut elements in the matrix : 55347
 
Starting tree search.
Deterministic mode with up to 32 running threads and up to 64 tasks.
Heap usage: 31MB (peak 59MB, 4394KB system)
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
       1  220494.0000  50297.62595      5      2      1   77.19%     779     13
       2  220494.0000  

    6654  140540.0000  71784.20292     13    945     12   48.92%     314     44
    6755  140540.0000  71784.20292     13    955     18   48.92%     563     45
    6855  140540.0000  71784.20292     13    936     36   48.92%     556     45
    6956  140540.0000  71784.20292     13    948     17   48.92%     460     45
    7059  140540.0000  71784.20292     13    934     27   48.92%     260     45
    7162  140540.0000  71784.20292     13    925     25   48.92%     464     46
    7262  140540.0000  71784.20292     13    945     35   48.92%     508     46
    7362  140540.0000  75372.98749     13    966     25   46.37%     523     46
    7465  140540.0000  75372.98749     13    959     15   46.37%     454     46
    7568  140540.0000  75372.98749     13    956     17   46.37%     431     47
    7671  140540.0000  78278.91962     13    956      9   44.30%     324     47
    7772  140540.0000  78278.91962     13    929     28   44.30%     369     47
    7875  140540.0000  78278.91962     1

#### Load libraries to run Classification model

In [10]:
import numpy as np
import sys
import pickle
import re
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from termcolor import colored
import xpress as xp
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,precision_recall_curve, confusion_matrix, accuracy_score
import xgboost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_curve
from sklearn import svm

#### Load features for classification

In [11]:
def feature_loader(set_path): 
    f0=pickle.load(open(set_path+'/alfa_f','rb'))
    f1=pickle.load(open(set_path+'/B_f_2','rb'))
    f2=pickle.load(open(set_path+'/B_f_4','rb'))
    f3=pickle.load(open(set_path+'/B_f_8','rb'))
    f4=pickle.load(open(set_path+'/lp_kmedian','rb'))
    f5=pickle.load(open(set_path+'/min_f','rb'))
    f6=pickle.load(open(set_path+'/V_f_2','rb'))
    f7=pickle.load(open(set_path+'/V_f_4','rb'))
    f8=pickle.load(open(set_path+'/V_f_8','rb'))
    f9=pickle.load(open(set_path+'/yf','rb'))
    f10=pickle.load(open(set_path+'/Yf','rb'))
    f11=pickle.load(open(set_path+'/µf','rb'))
    f12=pickle.load(open(set_path+'/Yfp','rb'))
#     f13=pickle.load(open(set_path+'/degree','rb'))
    f14=pickle.load(open(set_path+'/T_f_2','rb'))
    f15=pickle.load(open(set_path+'/T_f_4','rb'))
    f16=pickle.load(open(set_path+'/T_f_8','rb'))
#     f17=pickle.load(open(set_path+'/min_c','rb'))
#     f18=pickle.load(open(set_path+'/f1','rb'))
#     f19=pickle.load(open(set_path+'/f2','rb'))
#     f20=pickle.load(open(set_path+'/lp_prime','rb'))
#     f21=pickle.load(open(set_path+'/NLP1','rb'))
    #f22=pickle.load(open(set_path+'/NILP','rb'))
    #f23= pickle.load(open(set_path+'/ga_ind','rb'))
    
    features=np.vstack([f0[:],f1[:],f2[:],f3[:],f4[:],f5[:],f6[:],f7[:],f8[:],f9[:],f10[:],f11[:],f12[:],f14[:],f15[:],f16[:]]);features=features.transpose()
    #features=np.vstack([f0[:],f1[:],f2[:],f3[:],f4[:],f5[:],f6[:],f7[:],f8[:],f9[:],f10[:],f11[:]]);features=features.transpose()
    #features=np.vstack([f4[:],f10[:],f7[:],f2[:]]);features=features.transpose()
    
    
    return(features)

#### Load ground truth for classification

In [13]:
path='data/'+modes[0]+'/V='+str(nV[0])+'-C='+str(nC[0])+'/k='+str(K[0])+'/training/0'
y=pickle.load(open(path+'/points_inf_xp','rb'))
y_true=np.zeros(nV[0]);y_true[y[0][0]]=1
features=feature_loader(path)


for Mode in modes:
    for nv in nV:
        for nc in nC:
            for k in K:
                for i in range(1,num_graphs):
                    Mypath='data/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/training/'+str(i)
                    y=pickle.load(open(Mypath+'/points_inf_xp','rb'))
                    y_temp=np.zeros(nV[0]);y_temp[y[0][0]]=1
                    y_true=np.hstack([y_true,y_temp])
                    f_temp=feature_loader(Mypath)
                    features=np.vstack([features,f_temp])
print(len(y_true),len(features))


500 500


#### Normalisation and Learning Classification Model

In [20]:
sc = StandardScaler()
features = sc.fit_transform(features)


clf = RandomForestClassifier(max_depth=5,n_estimators=100,min_samples_split=10, random_state=0,class_weight="balanced")
# clf = xgboost.XGBClassifier(n_estimators=100, learning_rate=1.0, max_depth=3, random_state=0, class_weight='balanced')
# clf=AdaBoostClassifier(n_estimators=100, random_state=0)
# clf=svm.SVC(cache_size=200, class_weight='balanced',random_state=None)

clf.fit(features,y_true)

train_model_file = 'data/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/train_model_file'
pickle.dump(clf,open(train_model_file,'wb'))

print(colored('Feature importance:','blue'))
print(clf.feature_importances_)

Feature importance:
[0.18399594 0.08689538 0.01925897 0.01159508 0.01720737 0.03841272
 0.02608547 0.01009784 0.01596755 0.18049206 0.03889902 0.16991019
 0.05737471 0.04365763 0.04558098 0.0545691 ]


In [15]:
train_model_file = 'data/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/train_model_file'
clf_100=pickle.load(open(train_model_file,'r'))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [21]:
import numpy as np
import random
import pickle
import os

K=[10]
nV=[200]                      #nv is the number of vertices in the graph.  
nC=[20]                      #cv is the number of clients for each facility
num_graphs=1
penalty=10000

modes=['skewed[1-10]']
da = ['testing']
for nv in nV:
    for nc in nC:
        for mode in modes:
            os.system('mkdir'+' data/')
            os.system('mkdir'+' data/'+mode)
            os.system('mkdir'+' data/'+mode+'/V='+str(nv)+'-C='+str(nc))
            for ki,k in enumerate(K):
                os.system('mkdir'+' data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k))
                os.system('mkdir'+' data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/testing')
                print(nv,nc,mode,k)
                for d in da:
                    path='data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/'+d+'/'
                    for graph in range(num_graphs):
                        adj=np.zeros([nv*nc,3])
                        new_pairs=[]
                        for i in range(1,nv+1):
                            y=np.sort(random.sample(range(1,nv+1), nc))
                            for j in (y):
                                new_pairs.append((i,j))
                        new_pairs=np.array(new_pairs)


                        adj[:,0]=new_pairs[:,0]
                        adj[:,1]=new_pairs[:,1]
                        #adj[:,2]= random.choices([1,10], weights = [3, 6], k = nc*nv)
                        #adj[:,2]np.random.randint(low = 1,high=6, size=nc*nv)
                        adj[:,2]= random.choices([1,2,3,4,5,6,7,8,9,10], weights = [1,1,2,2,3,3,4,4,5,5], k = nc*nv)
                        #adj[:,2]= random.choices([1,100], k = nc*nv)

                        os.system("mkdir "+path+str(graph)); pickle.dump([adj,nv,k],open(path+str(graph)+'/data','wb'))
                        np.savetxt(path+str(graph)+'/data.txt', adj, fmt='%10d' ,header=str(nv)+'   '+str(nv*nc)+'   '+str(k))

200 20 skewed[1-10] 10


In [22]:
import xpress as xp
from termcolor import colored
import timeit
import numpy as np
import pickle
import random
def kmedian_xpress_solver_LP(set_path,data,c,n,k):
    print("Inside kmedian",n,k)
    S1 = timeit.default_timer()
    x = np.array ([xp.var (vartype = xp.continuous,ub=1.0) for i in range(n*n)]).reshape (n,n)
    y = np.array([xp.var (vartype = xp.continuous,ub=1.0) for i in range(n)])
    #x = np.array ([xp.var (vartype = xp.free) for i in range(n*n)]).reshape (n,n)
    #y = np.array([xp.var (vartype = xp.free) for i in range(n)])
    print(x.shape,y.shape)
    
    c1=[xp.Sum(y)==k]
    c2=xp.Dot(x,np.ones([n,1]))==np.ones([n,1])
    c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,n),x],axis=0))<=np.zeros([n,n])

    #c4=0<=x; c5=x<=1;  c6=0<=y;  c7=y<=1

    profit=xp.Sum(xp.Sum(c*x))

    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.minimize)
    try:
        p.addConstraint(c1,c2,c3)#,c4,c5,c6,c7)
    except xp.ModelError as e:
      print ("Modeling error:", repr(e))
    p.solve()
    y=p.getSolution(y)
    x=p.getSolution(x)
    E1 = timeit.default_timer()
    #################################################################################################
    median_indices=np.where(y>0.0)[0]
    print(colored('            Time: ','red'), E1 - S1) 
    print(colored('            objective value: ','red'),p.getSolution(profit))
    #print(colored('median indices: ','red'),median_indices+1,median_indices.shape)
     #------------------------------------------------------
    pickle.dump([median_indices,p.getSolution(profit),E1-S1],open(set_path+'/lp_inf','wb'))
    return median_indices,x,y

In [23]:
def features_related_LP(set_path,median_indices,x,y,c,n):
    #------------------lp_kmedian----------------------
    ind=np.zeros(n);ind[median_indices]=1
    pickle.dump(ind,open(set_path+'/lp_kmedian','wb'))
    pickle.dump(y,open(set_path+'/yf','wb'))
    #--------------------alfa_f-------------------------
    alfa_f=np.zeros(n)
    for i in range(n):
        alfa_f[i]= sum(x[:,i])
    pickle.dump(alfa_f,open(set_path+'/alfa_f','wb'))
    #---------------------µf-------------------------
    µf=np.zeros(n);
    for i in range(n):
        if alfa_f[i]!=0:
            µf[i]= sum(x[:,i]*c[:,i])/alfa_f[i]
    pickle.dump(µf,open(set_path+'/µf','wb'))
    #---------------------Vf,t(t=2,4,8)-------------------------
    T=[2,4,8]
    for t in T:
        V_ft=np.zeros(n);
        for i in range(n):
            V_ft[i]=len(np.where(c[:,i]<=(t*µf[i]))[0])
        pickle.dump(V_ft,open(set_path+'/V_f_'+str(t),'wb'))
    #---------------------Bf,t(t=2,4,8)-------------------------
    dav=np.zeros(n)
    for i in range(n):
        dav[i]=sum(c[:,i]*x[:,i])

    T=[2,4,8]
    for t in T:
        B_ft=np.zeros(n)
        for i in range(n):
            B_ft[i]=len(np.where(c[:,i]<=(t*dav[i]))[0])
        pickle.dump(B_ft,open(set_path+'/B_f_'+str(t),'wb'))
                    
    #----------------------------Yf----------------------------
    Yf=np.zeros(n)
    for i in range(n):
        Yf[i]=(len(np.where(x[:,i]!=0)[0]))
    pickle.dump(Yf,open(set_path+'/Yf','wb'))

In [24]:
def features_no_related_LP(Mypath,cost_mtr,nv):
    #--------------------------------------------------------
    degree=np.zeros(nv)
    for j in range(nv):
        s=np.where(cost_mtr[:,j]!=penalty)[0]
        degree[j]=sum(cost_mtr[s,j])
    pickle.dump(degree,open(Mypath+'/degree','wb'))
    #--------------------------------------------------------
    T=[2,4,8]
    for t in T:
        T_ft=np.zeros(nv);
        for i in range(nv):
            T_ft[i]=len(np.where(cost_mtr[:,i]<=t)[0])

        pickle.dump(T_ft,open(Mypath+'/T_f_'+str(t),'wb'))
    #--------------------------------------------------------
    min_c=np.zeros(nv)
    for i in range(nv):
        min_c[i]=min(cost_mtr[:,i])
    pickle.dump(min_c,open(Mypath+'/min_c','wb'))
    #---------------------------min_f-----------------------------
    min_f=np.zeros(nv)
    for i in range(nv):
        min_f[i]=min(cost_mtr[:,i])
    pickle.dump(min_f,open(Mypath+'/min_f','wb'))
    #----------------------------Yf,p=1.5----------------------------
    Yf=np.zeros(nv)
    for i in range(nv):
        Yf[i]=len(np.where(cost_mtr[:,i]<=(1.5*min_f[i]))[0])
    pickle.dump(Yf,open(Mypath+'/Yfp','wb'))
    #------------------------------------------------------------------------------------
    s=np.sort(random.sample(range(nv), int(nv/2)))
    s1=np.zeros(nv);s1[s]=1
    ss=[]
    for i in range(nv):
        if i not in s:
            ss.append(i)
    ss=np.array(ss);s2=np.zeros(nv);s2[ss]=1
    pickle.dump(s1,open(Mypath+'/f1','wb'))
    pickle.dump(s2,open(Mypath+'/f2','wb'))

In [25]:
for Mode in modes:
    print(Mode,'--------------------------')
    for nv in nV:
        for nc in nC:
            print('------------------------------------------')
            for k in K:
                print('------------------------------------------')
                for d in da:
                    print('--- nV=',nv,' nC=',nc,' k=',k, ' mode=',d)
                    for graph in range(num_graphs):
                        print('       data=',graph)
                        Mypath='data/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/'+d+'/'+str(graph)
                        data = pickle.load(open(Mypath+'/data','rb'))
                        nv= data[1]; k=data[2]; data=data[0]
                        data=data.astype(int)
                        data[:,0]=data[:,0]-1
                        data[:,1]=data[:,1]-1
                        cost_mtr=np.zeros([nv,nv])+penalty
                        for i in range (len(data)):
                            cost_mtr[data[i,1],data[i,0]]=data[i,2]
                        #print("Calling kmedian with ",Mypath,data.shape,cost_mtr.shape,nv,k)
                        median_indices,x,y=kmedian_xpress_solver_LP(Mypath,data,cost_mtr,nv,k)
                        features_related_LP(Mypath,median_indices,x,y,cost_mtr,nv)
                        features_no_related_LP(Mypath,cost_mtr,nv)

skewed[1-10] --------------------------
------------------------------------------
------------------------------------------
--- nV= 200  nC= 20  k= 10  mode= testing
       data= 0
Inside kmedian 200 10
(200, 200) (200,)
FICO Xpress v8.10.0, Hyper, solve started 15:27:13, Aug 13, 2021
Heap usage: 19MB (peak 19MB, 1640KB system)
Minimizing LP noname
Original problem has:
     40201 rows        40200 cols       120200 elements
Presolved problem has:
     40201 rows        40200 cols       120200 elements
Presolve finished in 0 seconds
Heap usage: 26MB (peak 40MB, 1641KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+01] / [ 1.00e+00,  1.00e+01]
  Objective      [min,max] : [ 1.00e+00,  1.00e+04] / [ 1.00e+00,  1.00e+04]
Autoscaling applied standard scaling

Crash basis containing 200 structural columns created
Starting paral

#### Output the learnt model to a file

In [26]:
for Mode in modes:
    for nv in nV:
        for nc in nC:
            for k in K:
                for i in range(num_graphs):
                    test_path = 'data/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/testing/'+str(i)

                    test_features=feature_loader(test_path)
                    test_features = sc.transform(test_features)

                    y_pred = clf.predict(test_features)

                    probs= clf.predict_proba(test_features)

                    pickle.dump(np.where(y_pred==1)[0],open(test_path+'/pruning_points_'+str(i),'wb'))
                    pickle.dump(probs[:,1],open(test_path+'/probs','wb'))
                    pickle.dump(np.where(y_pred==1)[0],open(test_path+'/probs_xgb','wb'))


                    print(len(np.where(y_pred==1)[0]))

10


#### Compute the ILP on pruned data set (pruning based on different thresholds)

In [31]:
import numpy as np
import sys
import pickle
import re
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from termcolor import colored
import xpress as xp
from sklearn.metrics import confusion_matrix
import random


def kmedian_xpress_solver_pruned(points,n,c,num_fac,k):
    S=range(n);SS=range(n*num_fac)
    x = np.array ([xp.var (vartype = xp.binary) for i in SS]).reshape (n,num_fac)
    y = np.array([xp.var (vartype = xp.binary) for i in range(num_fac)])

    c1=[xp.Sum(y)==k]
    c2=xp.Dot(x,np.ones([num_fac,1]))==np.ones([n,1])
    c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,num_fac),x],axis=0))<=np.zeros([n,num_fac])

    profit=xp.Sum(xp.Sum((c*x)))
    
    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.minimize)
    p.addConstraint(c1,c2,c3)
    p.solve()
    
    #print(colored('--------------','green'))
    #print(p.getSolution(y))
    #print(colored('--------------','green'))
    
    median_indices=np.where(p.getSolution(y)>=0.5)[0]
    #print(p.getSolution(y),k)
    #print(median_indices)

    return median_indices,p.getSolution(profit)


#///////////////////////////////////////////////////////////////////////////////////////////////////
for nv in nV:
    for nc in nC:
        for Mode in modes:
            for k in K:
                path='data/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/testing/'
                #thre = [k,min(int(1.5*k),nv),min(int(2*k),nv),min(int(3*k),nv),min(int(4*k),nv),min(int(5*k),nv),min(int(6*k),nv)]
                thre = [min(int(5*k),nv),min(int(6*k),nv)]
                value_n=np.zeros(len(thre))
                Time=np.zeros(len(thre))
                for m in range(num_graphs):
                    print(colored('....................................................','blue'),m)
                    set_path=path+str(m)
                    data = pickle.load(open(set_path+'/data','rb'))

                    probs=pickle.load(open(set_path+'/probs','rb'))
                    ind=np.argsort(probs)


                    n=data[1];k=data[2]
                    data=data[0].astype(int)
                    data[:,0]=data[:,0]-1
                    data[:,1]=data[:,1]-1

                    for th_index,th in enumerate(thre):
                        ind_last_thre = ind[len(ind)-th:]
                        num_fac = len(ind_last_thre)

                        c=np.zeros([n,len(ind_last_thre)])
                        for i in range(n):
                            for j in range(len(ind_last_thre)):
                                    c[i,j]=penalty

                        for i in range (len(ind_last_thre)):
                            mp=(np.where(data[:,0]==ind_last_thre[i])[0])
                            c[data[mp,1],i]=data[mp,2]

                        #print(colored("--------------------",'blue'))
                        #print(data,n,c,num_fac,k)
                        #print(colored("--------------------",'blue'))
                        S=timer()
                        median_indices,value=kmedian_xpress_solver_pruned(data,n,c,num_fac,k)
                        E=timer()
                        print(colored('pruned centers:','red'),ind_last_thre+1)
                        print(colored('selected centers:','red'),np.sort(ind_last_thre[(median_indices)])+1,len(median_indices))
                        print(colored('Objective Value:','red'),value)
                        print(E-S)

                        #s=pickle.load(open(set_path+'/points_inf_xp','rb'))
                        #s=s[0][1]
                        value_n[th_index]=value
                        Time[th_index]=E-S
                        #print(s,value_n[thi],value)
                    print(colored("Value_n: " +str(value_n),'red'))
                    print(colored("Time: "+str(Time),'red'))

.................................................... 0
FICO Xpress v8.10.0, Hyper, solve started 17:09:57, Aug 13, 2021
Heap usage: 5453KB (peak 5453KB, 1149KB system)
Minimizing MILP noname
Original problem has:
     10201 rows        10050 cols        30050 elements     10050 globals
Presolved problem has:
     10201 rows        10050 cols        30050 elements     10050 globals
Presolve finished in 0 seconds
Heap usage: 8651KB (peak 12MB, 1150KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+01] / [ 1.00e+00,  1.00e+01]
  Objective      [min,max] : [ 1.00e+00,  1.00e+04] / [ 1.00e+00,  1.00e+04]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 50.4GB
 *** Heuristic solution found:  1800142.000      Time: 0 ***
Starting concurrent solve with dual, primal and barrier (30 thread


Reduced problem has:   10219 rows   10050 columns     38351 elements
Presolve dropped   :       0 rows       0 columns     50251 elements
Will try to keep branch and bound tree memory usage below 51.2GB
Crash basis containing 50 structural columns created
 
   Its         Obj Value      S   Ninf  Nneg        Sum Inf  Time
1988189       174870.4725      P      0     0        .000000    69
Optimal solution found
Dual solved problem
  1988189 simplex iterations in 69s

Final objective                       : 1.748704725383414e+05
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max dual violation        (abs/rel) :       0.0 /       0.0
  Max complementarity viol. (abs/rel) :       0.0 /       0.0
 
Starting root cutting & heuristics
 
 Its Type    BestSoln    BestBound   Sols    Add    Del     Gap     GInf   Time
   1  K   460954.0000  252280.4981     10      7      0   45.27%    1128     71
   2  K   460954.0000  252280.4981     10      5      8   45.27%    1118     72
 

    8611  431031.0000  252280.4981     13   3145     16   41.47%     731    214
    8711  431031.0000  252280.4981     13   3167     24   41.47%     742    215
    8814  431031.0000  252280.4981     13   3201     32   41.47%     825    216
    8915  431031.0000  252280.4981     13   3209     35   41.47%     771    216
    9018  431031.0000  252280.4981     13   3250     17   41.47%     592    217
    9120  431031.0000  252280.4981     13   3276     35   41.47%     641    217
Heap usage: 1994MB (peak 1994MB, 76MB system)
B&B tree size: 124MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    9220  431031.0000  252280.4981     13   3330     28   41.47%     772    218
    9322  431031.0000  252280.4981     13   3363     23   41.47%     555    218
    9423  431031.0000  252280.4981     13   3386     31   41.47%     926    219
    9524  431031.0000  252280.4981     13   3409     18   41.47%     752    219
    9629  431031.0000  252280.4981     13   3

R         540954.0000  146031.2336      4                 73.00%       0      1
R         540887.0000  146031.2336      5                 73.00%       0      2
   1  K   540887.0000  146031.2336      5     13      0   73.00%    1324      2
   2  K   540887.0000  147977.0925      5      5      7   72.64%    1265      3
   3  K   540887.0000  148630.8268      5      5      5   72.52%    1281      3
   4  K   540887.0000  149020.0210      5      4      4   72.45%    1257      3
   5  K   540887.0000  149437.3342      5      4      4   72.37%    1271      4
   6  K   540887.0000  150125.9803      5      4      5   72.24%    1273      4
   7  K   540887.0000  150327.1916      5      4      5   72.21%    1231      5
   8  K   540887.0000  150457.7796      5      3      4   72.18%    1283      5
   9  K   540887.0000  150487.1511      5      3      1   72.18%    1298      5
  10  K   540887.0000  150491.4138      5      6      3   72.18%    1311     26
  11  K   540887.0000  150496.1679      

    3516  461064.0000  168649.4974      7      7      6   63.42%    1170    147
    3517  461064.0000  168649.4974      7      8      6   63.42%    1194    147
    3527  461064.0000  182261.7384      7     13      6   60.47%    1166    149
    3537  461064.0000  192022.7793      7     20      8   58.35%    1069    149
    3547  461064.0000  192022.7793      7     25      8   58.35%    1170    150
    3557  461064.0000  192022.7793      7     32     10   58.35%    1242    151
    3567  461064.0000  195779.6542      7     40      7   57.54%     896    151
    3577  461064.0000  208285.0695      7     47      9   54.83%    1143    152
    3587  461064.0000  208285.0695      7     53      8   54.83%     954    152
    3597  461064.0000  208285.0695      7     60      7   54.83%     857    153
    3607  461064.0000  208285.0695      7     69      9   54.83%     975    153
Heap usage: 1063MB (peak 1069MB, 39MB system)
B&B tree size: 12MB total
 
    Node     BestSoln    BestBound   Sols Acti

   11674  450920.0000  221546.7996     10   4709     36   50.87%     599    313
   11775  450920.0000  221546.7996     10   4763     27   50.87%     963    314
   11875  450920.0000  221546.7996     10   4798     30   50.87%     717    314
   11976  450920.0000  221546.7996     10   4858     19   50.87%     912    315
   12076  450920.0000  221546.7996     10   4942     22   50.87%     853    315
   12177  450920.0000  221546.7996     10   4966     28   50.87%     741    316
   12280  450920.0000  221546.7996     10   4975     48   50.87%     802    316
   12380  450920.0000  221546.7996     10   5033     14   50.87%    1030    316
   12480  450920.0000  221546.7996     10   5070     31   50.87%     911    317
   12580  450920.0000  221546.7996     10   5152     31   50.87%     688    317
   12681  450920.0000  221546.7996     10   5205     18   50.87%     739    318
   12786  450920.0000  221546.7996     10   5259     40   50.87%     736    318
   12886  450920.0000  221546.7996     1

#### Add a constraint that at least 80% of nodes to be taken from pruned set

In [30]:
import numpy as np
import sys
import pickle
import re
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from termcolor import colored
import xpress as xp
from sklearn.metrics import confusion_matrix
import random


def kmedian_xpress_solver_constrained(points,n,c,fac_index,k):
    S=range(n);SS=range(n*n)
    x = np.array ([xp.var (vartype = xp.binary) for i in SS]).reshape (n,n)
    y = np.array([xp.var (vartype = xp.binary) for i in range(n)])

    c1=[xp.Sum(y)==k]
    c2=xp.Dot(x,np.ones([n,1]))==np.ones([n,1])
    c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,n),x],axis=0))<=np.zeros([n,n])
    c4=[xp.Sum(y[fac_index])>=int(0.8*k)]

    profit=xp.Sum(xp.Sum((c*x)))
    
    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.minimize)
    p.addConstraint(c1,c2,c3,c4)
    p.solve()
    
    #print(colored('--------------','green'))
    #print(p.getSolution(y))
    #print(colored('--------------','green'))
    
    median_indices=np.where(p.getSolution(y)>=0.5)[0]
    #print(p.getSolution(y),k)
    #print(median_indices)

    return median_indices,p.getSolution(profit)


#///////////////////////////////////////////////////////////////////////////////////////////////////
for nv in nV:
    for nc in nC:
        for Mode in modes:
            for k in K:
                path='data/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/testing/'
                thre = [k,min(int(1.5*k),nv),min(int(2*k),nv),min(int(3*k),nv),min(int(4*k),nv)]
                value_n=np.zeros(len(thre))
                Time=np.zeros(len(thre))
                for m in range(num_graphs):
                    print(colored('....................................................','blue'),m)
                    set_path=path+str(m)
                    data = pickle.load(open(set_path+'/data','rb'))

                    probs=pickle.load(open(set_path+'/probs','rb'))
                    ind=np.argsort(probs)


                    n=data[1];k=data[2]
                    data=data[0].astype(int)
                    data[:,0]=data[:,0]-1
                    data[:,1]=data[:,1]-1

                    for th_index,th in enumerate(thre):
                        ind_last_thre = ind[len(ind)-th:]

                        cost_mtr=np.zeros([n,n])+penalty
                        for i in range (len(data)):
                            cost_mtr[data[i,1],data[i,0]]=data[i,2]

                        #print(colored("--------------------",'blue'))
                        #print(data,n,c,num_fac,k)
                        #print(colored("--------------------",'blue'))
                        S=timer()
                        median_indices,value=kmedian_xpress_solver_constrained(data,n,cost_mtr,ind_last_thre,k)
                        E=timer()
                        print(colored('pruned centers:','red'),ind_last_thre+1)
                        print(colored('selected centers:','red'),np.sort(median_indices)+1,len(median_indices))
                        print(colored('Objective Value:','red'),value)
                        print(E-S)

                    
                        value_n[th_index]=value
                        Time[th_index]=E-S
                        #print(s,value_n[thi],value)
                    print(colored("Value_n: " +str(value_n),'red'))
                    print(colored("Time: "+str(Time),'red'))

.................................................... 0
FICO Xpress v8.10.0, Hyper, solve started 15:33:38, Aug 13, 2021
Heap usage: 20MB (peak 20MB, 1656KB system)
Minimizing MILP noname
Original problem has:
     40202 rows        40200 cols       120210 elements     40200 globals
Presolved problem has:
     40202 rows        40200 cols       120210 elements     40200 globals
Presolve finished in 0 seconds
Heap usage: 33MB (peak 49MB, 1657KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+01] / [ 1.00e+00,  1.00e+01]
  Objective      [min,max] : [ 1.00e+00,  1.00e+04] / [ 1.00e+00,  1.00e+04]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 58.5GB
 *** Heuristic solution found:  1800132.000      Time: 1 ***
Starting concurrent solve with dual, primal and barrier (30 threads)

  

   4  K   530922.0000  344877.4554      8      8      4   35.04%     876      7
   5  K   530922.0000  345114.6126      8      6      9   35.00%     869      8
   6  K   530922.0000  345236.5557      8      7      5   34.97%     945      8
   7  K   530922.0000  345377.6600      8      5      5   34.95%     928      9
q         500951.0000  345540.2745      9                 31.02%       0     10
   8  K   500951.0000  345540.2745      9      6      7   31.02%     935     10
   9  K   500951.0000  345675.0347      9      9      4   31.00%     863     10
R         500949.0000  345675.0347     10                 31.00%       0     11
R         500933.0000  345675.0347     11                 30.99%       0     11
  10  K   500933.0000  345675.0347     11      0     11   30.99%     948     12
Heuristic search started
Heuristic search stopped
 
Cuts in the matrix         : 9
Cut elements in the matrix : 85006
 
Starting tree search.
Deterministic mode with up to 32 running threads and up to

   9  K   620841.0000  302653.7645      6      5      4   51.25%    1079     10
  10  K   620841.0000  302656.4091      6      1      4   51.25%    1061     16
  11  K   620841.0000  302656.4091      6      0      1   51.25%    1024     17
Heuristic search started
Heuristic search stopped
 
Cuts in the matrix         : 11
Cut elements in the matrix : 138315
 
Starting tree search.
Deterministic mode with up to 32 running threads and up to 64 tasks.
Heap usage: 110MB (peak 162MB, 3295KB system)
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
       1  620841.0000  302997.5896      6      2      1   51.20%    1024     33
       2  620841.0000  309875.6056      6      2      3   50.09%     990     35
       3  620841.0000  315226.5825      6      4      3   49.23%     890     36
       4  620841.0000  318702.1248      6      4      4   48.67%    1063     37
       5  620841.0000  318702.1248      6      5      4   48.67%     890     38
       6  620841.00

b         570906.0000  183604.3843      4                 67.84%       0      3
R         570868.0000  183604.3843      5                 67.84%       0      5
   1  K   570868.0000  183604.3843      5     10      0   67.84%    1371      5
d         560935.0000  184812.6792      6                 67.05%       0      5
   2  K   560935.0000  184812.6792      6      6      7   67.05%    1393      5
   3  K   560935.0000  186085.4728      6      5      4   66.83%    1356      6
   4  K   560935.0000  186634.4275      6      4      6   66.73%    1272      7
   5  K   560935.0000  187508.1694      6      4      3   66.57%    1282      8
d         550935.0000  188236.2046      7                 65.83%       0      9
   6  K   550935.0000  188236.2046      7      3      2   65.83%    1313      9
   7  K   550935.0000  188590.3120      7      3      3   65.77%    1265     10
   8  K   550935.0000  188741.9892      7      3      3   65.74%    1344     11
   9  K   550935.0000  188755.2652      

Heuristic search started
Heuristic search stopped
 
Cuts in the matrix         : 12
Cut elements in the matrix : 187584
 
Starting tree search.
Deterministic mode with up to 32 running threads and up to 64 tasks.
Heap usage: 116MB (peak 5526MB, 3314KB system)
Heap usage: 116MB (peak 5526MB, 3314KB system)
B&B tree size: 113k total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    3082  461008.0000  238265.8967     17      2      1   48.32%    1376    394
    3083  461008.0000  238265.8967     17      2      3   48.32%    1346    398
    3084  461008.0000  238265.8967     17      2      3   48.32%    1308    398
    3085  461008.0000  238265.8967     17      3      4   48.32%    1413    399
    3086  461008.0000  238265.8967     17      4      4   48.32%    1333    400
    3087  461008.0000  238265.8967     17      5      5   48.32%    1192    401
    3088  461008.0000  238265.8967     17      5      4   48.32%    1386    401
    3089  461008.0000  23

    9883  421000.0000  267680.6580     22   2245     20   36.42%     833   1122
    9983  421000.0000  267680.6580     22   2261     20   36.42%     845   1127
   10083  421000.0000  267680.6580     22   2273     18   36.42%     845   1133
   10183  421000.0000  267973.7778     22   2273     18   36.35%     839   1140
   10284  421000.0000  267973.7778     22   2283     13   36.35%     573   1147
   10384  421000.0000  267973.7778     22   2285     30   36.35%     820   1152
   10484  421000.0000  267973.7778     22   2294     19   36.35%     817   1157
   10585  421000.0000  267973.7778     22   2302     25   36.35%     826   1164
   10685  421000.0000  267973.7778     22   2308     20   36.35%     808   1172
   10785  421000.0000  267973.7778     22   2314     17   36.35%     726   1180
Heap usage: 6958MB (peak 6958MB, 81MB system)
B&B tree size: 0.5GB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
   10887  421000.0000  267973.7778     22   2

       2  540890.0000  133882.7244      4      3      3   75.25%    1687     54
       3  540890.0000  133882.7244      4      4      3   75.25%    1537     57
       4  540890.0000  135153.6892      4      5      4   75.01%    1786     59
       5  540890.0000  135153.6892      4      5      4   75.01%    1590     62
       6  540890.0000  135153.6892      4      6      4   75.01%    1547     65
       7  540890.0000  135153.6892      4      8      4   75.01%    1311     65
       8  540890.0000  135153.6892      4      8      5   75.01%    1593     66
       9  540890.0000  135153.6892      4      9      5   75.01%    1708     68
      10  540890.0000  137456.9866      4     10      5   74.59%    1620     69
      20  540890.0000  142083.9515      4     20      6   73.73%    1551     76
      30  540890.0000  149187.3848      4     27      7   72.42%    1548     78
      40  540890.0000  149237.1653      4     34      8   72.41%    1482     81
      50  540890.0000  151978.4281      

    3379  470994.0000  172915.3401      9    262     13   63.29%     807    569
    3479  470994.0000  172915.3401      9    362     16   63.29%    1208    581
    3579  470994.0000  172915.3401      9    462     20   63.29%    1370    592
    3679  470994.0000  172915.3401      9    562     22   63.29%     886    602
    3779  470994.0000  172915.3401      9    662     25   63.29%    1204    611
b   3851  470973.0000  172915.3401     10    735     32   63.29%       0    617
    3879  470973.0000  172915.3401     10    762     32   63.29%     419    619
    3979  470973.0000  172915.3401     10    857     27   63.29%    1163    627
b   3986  441016.0000  172915.3401     11    864     38   60.79%       0    627
    4079  441016.0000  172915.3401     11    919     32   60.79%    1478    635
    4179  441016.0000  172915.3401     11   1004     35   60.79%    1053    645
    4279  441016.0000  172915.3401     11   1091     36   60.79%    1150    655
    4379  441016.0000  172915.3401     1

   12303  401006.0000  182291.1866     18   3971     17   54.54%     760   1433
   12403  401006.0000  182291.1866     18   4005     16   54.54%    1297   1443
   12503  401006.0000  182291.1866     18   4027     21   54.54%     724   1453
   12604  401006.0000  182291.1866     18   4076     26   54.54%     890   1461
Heap usage: 7077MB (peak 7077MB, 81MB system)
B&B tree size: 0.5GB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
   12706  401006.0000  182291.1866     18   4091     31   54.54%     786   1469
   12806  401006.0000  182291.1866     18   4124     26   54.54%     948   1476
   12910  401006.0000  182291.1866     18   4154     30   54.54%     733   1483
   13010  401006.0000  182291.1866     18   4196     15   54.54%    1016   1492
   13110  401006.0000  182291.1866     18   4204     16   54.54%    1048   1501
   14110  401006.0000  182291.1866     18   4442     27   54.54%     647   1579
   15111  401006.0000  185333.0723     18   4